In [1]:
import pyptvgtfs
from pyptvgtfs import BRANCH_IDS, GTFS_FILE_FIELDS_TYPES, TABLE_NAMES, BRANCH_IDS_ALL
import pandas as pd
import os
import datetime as dt
import re
import plotly.graph_objs as go
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString, MultiLineString, Polygon, MultiPolygon, GeometryCollection

In [2]:
VERSIONS = [
    '20220403_025040',
    '20230805_030129',
    '20231021_105623',
    '20240229_224711'
]

# VERSIONS x BRANCHES
VERSIONS_BRANCHES = [(v, b) for v in VERSIONS for b in BRANCH_IDS]
VERSIONS_BRANCHES_ALL = [(v, b) for v in VERSIONS for b in BRANCH_IDS_ALL]

In [3]:
DFS_LIST = [pyptvgtfs.process_gtfs_zip(f'../downloads/{f}/gtfs.zip', f) for f in VERSIONS]
# Per file: 40s - 1m - 3m. 5 files: 2m - 5m. 7 files: 3m - 5sm. 2 files: 1m 30s - 2m

DFK : dict[tuple, pd.DataFrame] = pd.concat(DFS_LIST, axis=0).set_index(['version_id', 'branch_id', 'table_name'])['df'].to_dict()

DF : dict[str, dict[str, dict[str, pd.DataFrame]]] = {}
for (vid, bid, table_name), df in DFK.items():
    DF[vid] = DF.get(vid, {})
    DF[vid][bid] = DF[vid].get(bid, {})
    DF[vid][bid][table_name] = df


# Either route_code_extra or route_no is unique, and the other denotes the variants of the same route
# If both are not unique, then the route_no is the variant of the route, while the route_code_extra should also belongs to route_code
for vid, bid in VERSIONS_BRANCHES:
    df_route_idx = DF[vid][bid]['route_ids']['route_id'].apply(lambda x: x.split('-'))
    df_route_code_extra = df_route_idx.apply(lambda x: x[2] if len(x) >= 5 else '')
    df_route_no = df_route_idx.apply(lambda x: x[-1])
    if not (df_route_code_extra.nunique() == 1 or df_route_no.nunique() == 1):
        assert bid == '4'
        

for vid, bid in VERSIONS_BRANCHES:
    DF[vid][bid]['service_ids'] = pd.DataFrame(DF[vid][bid]['calendar']['service_id'].drop_duplicates().reset_index(drop=True))
    DF[vid][bid]['route_ids'] = pd.DataFrame(DF[vid][bid]['routes']['route_id'].drop_duplicates().reset_index(drop=True))
    DF[vid][bid]['trip_ids'] = pd.DataFrame(DF[vid][bid]['trips']['trip_id'].drop_duplicates().reset_index(drop=True))
    DF[vid][bid]['shape_ids'] = pd.DataFrame(DF[vid][bid]['shapes']['shape_id'].drop_duplicates().reset_index(drop=True))
# 1s - 2s

for vid, bid in VERSIONS_BRANCHES:
    # Get all types of delimiters
    DF[vid][bid]['patterns'] = {}
    DF[vid][bid]['patterns'] = {}
    DF[vid][bid]['delimiters'] = {}
    for id_name in ['service_id', 'route_id', 'trip_id', 'shape_id']:
        id_pattern = DF[vid][bid][f'{id_name}s'][id_name].str.replace(r'[a-zA-Z0-9]+', '0', regex=True).drop_duplicates()
        DF[vid][bid]['patterns'][id_name] = id_pattern.unique()
        id_pattern = id_pattern.str.replace(r'[0]', '', regex=True).unique()
        # Sum all in DF[vid][bid]['patterns'][id_name] and remove duplicates
        DF[vid][bid]['delimiters'][id_name] = set(''.join(id_pattern))
# 2s - 5s
        
ID_PATTERNS = {
    k: pd.DataFrame(
        data=[(vid, bid, DF[vid][bid]["patterns"][k]) for vid, bid in VERSIONS_BRANCHES],
        columns=["version_id", "branch_id", "pattern"],
    )
    .explode("pattern")
    .groupby("pattern")["branch_id"]
    .apply(lambda x: sorted(int(i) for i in x.unique()))
    .to_dict()
    for k in ["service_id", "route_id", "trip_id", "shape_id"]
}


ID_PATTERNS == {
    "service_id": {
        "0": [1, 2, 3, 4, 5, 6, 10, 11],
        "0+0": [1, 2, 3, 4, 5, 6, 10],
        "0+0_0": [1, 2, 3, 4, 5, 6, 10],
        "0-0-0-0": [4],
        "0-0-0-0-0": [4],
        "0_0": [1, 2, 3, 4, 5, 6, 10],
    },
    "route_id": {
        "0-0-0-0": [1, 2, 3, 4, 5, 6, 10, 11],
        "0-0-0-0-0": [1, 2, 3, 4, 5, 6, 10],
    },
    "trip_id": {
        "0-0--0-0-0": [4],
        "0-0-0-0-0-0": [4],
        "0.0.0-0-0-0-0.0.0": [1, 2, 3, 5, 6, 10],
        "0.0.0-0-0-0.0.0": [1, 2, 3, 5, 6, 10, 11],
    },
    "shape_id": {
        "0-0-0-0-0.0.0": [1, 2, 3, 4, 5, 6, 10],
        "0-0-0-0.0.0": [1, 2, 3, 4, 5, 6, 10, 11],
    },
}


for vid, bid in VERSIONS_BRANCHES:
    DF[vid][bid]['service_ids']['service_class'] = DF[vid][bid]['service_ids']['service_id'].apply(lambda x: x.split('-')[0].split('+')[0].split('_')[0])

for vid, bid in VERSIONS_BRANCHES:
    df_route_idx = DF[vid][bid]['route_ids']['route_id'].apply(lambda x: x.split('-'))
    DF[vid][bid]['route_ids']['route_code'] = df_route_idx.apply(lambda x: x[1] + x[2] if (bid == '4' and len(x) >= 5) else x[1])
    DF[vid][bid]['route_ids']['route_no'] = df_route_idx.apply(lambda x: x[-1])
    DF[vid][bid]['route_ids']['branch'] = df_route_idx.apply(lambda x: x[0])
    DF[vid][bid]['route_ids']['range'] = df_route_idx.apply(lambda x: x[-2])


for vid, bid in VERSIONS_BRANCHES:
    df_shape_idx = DF[vid][bid]['shape_ids']['shape_id'].apply(lambda x: x.split('.'))
    df_route_id = df_shape_idx.apply(lambda x: x[0])
    df_route_idx = df_route_id.apply(lambda x: x.split('-'))

    DF[vid][bid]['shape_ids']['route_id'] = df_route_id
    DF[vid][bid]['shape_ids']['route_code'] = df_route_idx.apply(lambda x: x[1] + x[2] if (bid == '4' and len(x) >= 5) else x[1])
    DF[vid][bid]['shape_ids']['route_no'] = df_route_idx.apply(lambda x: x[-1])
    DF[vid][bid]['shape_ids']['branch'] = df_route_idx.apply(lambda x: x[0])
    DF[vid][bid]['shape_ids']['direction'] = df_shape_idx.apply(lambda x: x[2])
    DF[vid][bid]['shape_ids']['range'] = df_route_idx.apply(lambda x: x[-2])
    DF[vid][bid]['shape_ids']['shape_no'] = df_shape_idx.apply(lambda x: x[1])
# 1s
    
for vid, bid in VERSIONS_BRANCHES:
    if bid == '4':
        df_trip_idx = DF[vid][bid]['trip_ids']['trip_id'].apply(lambda x: x.split('-'))
        DF[vid][bid]['trip_ids']['route_code'] = df_route_idx.apply(lambda x: x[1] + x[2] if (bid == '4' and len(x) >= 5) else x[1])
        DF[vid][bid]['trip_ids']['route_no'] = df_trip_idx.apply(lambda x: x[3])
        DF[vid][bid]['trip_ids']['branch'] = df_trip_idx.apply(lambda x: x[0])
        DF[vid][bid]['trip_ids']['service_class'] = df_trip_idx.apply(lambda x: x[4])
        DF[vid][bid]['trip_ids']['trip_no'] = df_trip_idx.apply(lambda x: x[5])
    else:
        df_trip_idx = DF[vid][bid]['trip_ids']['trip_id'].apply(lambda x: x.split('.'))
        df_route_id = df_trip_idx.apply(lambda x: x[2])
        df_route_idx = df_route_id.apply(lambda x: x.split('-'))

        # DF[vid][bid]['trip_ids']['route_id'] = df_route_id
        # DF[vid][bid]['trip_ids']['direction'] = df_trip_idx.apply(lambda x: x[4])
        # DF[vid][bid]['trip_ids']['shape_no'] = df_trip_idx.apply(lambda x: x[3])
        # DF[vid][bid]['trip_ids']['range'] = df_route_idx.apply(lambda x: x[-2])
        
        DF[vid][bid]['trip_ids']['route_code'] = df_route_idx.apply(lambda x: x[1] + x[2] if (bid == '4' and len(x) >= 5) else x[1])
        DF[vid][bid]['trip_ids']['route_no'] = df_route_idx.apply(lambda x: x[-1])
        DF[vid][bid]['trip_ids']['branch'] = df_route_idx.apply(lambda x: x[0])
        DF[vid][bid]['trip_ids']['service_class'] = df_trip_idx.apply(lambda x: x[1])
        DF[vid][bid]['trip_ids']['trip_no'] = df_trip_idx.apply(lambda x: x[0])
# 7s - 20s

for vid, bid in VERSIONS_BRANCHES:
    DF[vid][bid]['shapes'].sort_values(by=['shape_id', 'shape_pt_sequence'], inplace=True)
# 20s - 30s

for vid, bid in VERSIONS_BRANCHES:    
    DF[vid][bid]['shapes']['point'] = list(zip(DF[vid][bid]['shapes']['shape_pt_lon'], DF[vid][bid]['shapes']['shape_pt_lat']))
    # 1m - 2m. Occasionally 20s - 30s


for vid, bid in VERSIONS_BRANCHES:
    DF[vid][bid]['lines'] = DF[vid][bid]['shapes'].groupby('shape_id')[['shape_pt_sequence', 'point']].aggregate({'shape_pt_sequence': list, 'point': list}).reset_index()
    # 2m 30s. Occasionally 1m - 2m

# Optional: drop column
# for vid, bid in VERSIONS_BRANCHES:    
#     DF[vid][bid]['shapes'].drop(columns=['points'], inplace=True)
#     # 1m - 2m
    
# Maybe faster:
# for vid, bid in VERSIONS_BRANCHES:
#     DF[vid][bid]['lines'] = DF[vid][bid]['shapes'].groupby('shape_id')[['shape_pt_lon', 'shape_pt_lat']].apply(lambda x: list(zip(x['shape_pt_lon'], x['shape_pt_lat']))).reset_index(name='line')
#     # 2m - 3m

# Proof that all shape_pt_sequence are continuous
for vid, bid in VERSIONS_BRANCHES:
    assert DF[vid][bid]['lines']['shape_pt_sequence'].apply(lambda x: x[-1] - x[0] + 1 == len(x)).all(), (vid, bid)
    # 1s - 2s

for vid, bid in VERSIONS_BRANCHES:
    DF[vid][bid]['lines'] = pd.merge(DF[vid][bid]['lines'], DF[vid][bid]['shape_ids'], on='shape_id')

for vid, bid in VERSIONS_BRANCHES:
    DF[vid][bid]['lines']['geometry'] = DF[vid][bid]['lines']['point'].apply(lambda x: LineString(x))
    # 3m 30s - 5m


# Total from start: 13m - 22m

In [ ]:
def get_dates(monday, tuesday, wednesday, thursday, friday, saturday, sunday, start_date, end_date):
    # Get list of dates based on week pattern and date range
    week_pattern = [bool(int(monday)), bool(int(tuesday)), bool(int(wednesday)), bool(int(thursday)), bool(int(friday)), bool(int(saturday)), bool(int(sunday))]
    start_date = pd.to_datetime(start_date, format='%Y%m%d')
    end_date = pd.to_datetime(end_date, format='%Y%m%d')
    dates = pd.date_range(start_date, end_date)
    return dates[[week_pattern[i] for i in dates.dayofweek]]
    
def get_dates_df_calendar(df_calendar: pd.DataFrame, df_calendar_dates: pd.DataFrame):
    
    weekdate_columns = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
    daterange_columns = ['start_date', 'end_date']
    pattern_columns = weekdate_columns + daterange_columns
    
    # Drop duplicates to reduce the number of rows to be processed
    df_dates = df_calendar[pattern_columns].drop_duplicates()
    
    # Get date list based on week pattern and date range
    df_dates['date'] = df_dates.apply(lambda x: get_dates(x['monday'], x['tuesday'], x['wednesday'], x['thursday'], x['friday'], x['saturday'], x['sunday'], x['start_date'], x['end_date']), axis=1)

    df_dates['date'] = df_dates['date'].apply(lambda x: [y.strftime('%Y%m%d') for y in x])
    
    # Join the date list with the original calendar table
    df_dates = pd.merge(df_calendar, df_dates, on=pattern_columns, how='left')
    
    # Explode the date list into separate rows
    df_dates = df_dates[['service_id', 'date']].explode('date')

    # Join the date df with the calendar_dates df
    df_dates = pd.merge(df_dates, df_calendar_dates.astype({'date': str, 'exception_type': str}), on=['service_id', 'date'], how='outer')
    
    # Drop 2 and keep 1 and NaN
    df_dates = df_dates[df_dates['exception_type'] != '2'].reset_index(drop=True)

    return df_dates


for vid, bid in VERSIONS_BRANCHES:
    DF[vid][bid]['dates'] = get_dates_df_calendar(DF[vid][bid]['calendar'], DF[vid][bid]['calendar_dates'])
# 1s - 5s

for vid, bid in VERSIONS_BRANCHES:
    DF[vid][bid]['route_services'] = DF[vid][bid]['trips'][['route_id', 'service_id']].drop_duplicates().reset_index(drop=True)
    DF[vid][bid]['route_services'] = pd.merge(DF[vid][bid]['route_services'], DF[vid][bid]['route_ids'], on='route_id', how='left')
    DF[vid][bid]['route_services'] = pd.merge(DF[vid][bid]['route_services'], DF[vid][bid]['service_ids'], on='service_id', how='left')

for vid, bid in VERSIONS_BRANCHES:
    DF[vid][bid]['route_service_dates'] = pd.merge(DF[vid][bid]['route_services'], DF[vid][bid]['dates'], on='service_id', how='left')


In [ ]:
# It seems that there is no significant difference in shapes among different branches for each route_code. In each route, the shape with the most number of points among each branch is the same.
route_code = '742'
branch_codes = [19, 21, 22, 29]
branch_codes = [str(i) for i in branch_codes]
df903 = DF['20240229_224711']['4']['lines'][(DF['20240229_224711']['4']['lines']['route_code'] == route_code)].sort_values(by='point', key=lambda x: x.apply(lambda x: len(x)), ascending=False)
tl = {t:df903[df903['branch'] == t]['geometry'].iloc[0] for t in branch_codes}
for l in tl.values():
    plt.plot(*l.xy, linewidth=1)
plt.legend(branch_codes)

In [ ]:
for vid, bid in VERSIONS_BRANCHES:
    DF[vid][bid]['rd'] = DF[vid][bid]['route_service_dates'].groupby(['route_code', 'date'])['branch'].unique()
    # 1m 30s - 2m - 3m

# route_code-date pair may still have different branches
for vid in VERSIONS:
    print(DF[vid]['4']['rd'].apply(lambda x: len(x)).unique())

dfrd4 = DF[vid]['4']['route_service_dates'].copy(deep=True)
dfrd4['service_class_branch'] = dfrd4['service_class'] + '-' + dfrd4['branch'].astype(str)
dfrd4 = dfrd4.groupby(['route_code', 'date'])['service_class_branch'].unique().reset_index(name='service_class_branches')
dfrd4['len'] = dfrd4['service_class_branches'].apply(len)
dfrd4.sort_values(by='len', ascending=False, inplace=True)

dfsb4 = DF[vid]['4']['route_service_dates'][['service_class', 'branch']].drop_duplicates()
dfsb4 = dfsb4.groupby('branch')['service_class'].unique().reset_index(name='service_classes')

dfsb4['pattern'] = dfsb4['service_classes'].apply(lambda x: '-'.join(sorted(x)))

In [288]:
df4_trips = DF[vid]['4']['stop_times'].sort_values(by=['trip_id', 'stop_sequence'])
# 5s - 10s
df4_trips1 = df4_trips.groupby('trip_id')['stop_id'].apply(np.array).reset_index(name='stop_ids')
# 2s - 5s, using np.array. Faster than using list (2s - 5s)
df4_trips2 = df4_trips.groupby('trip_id')['stop_sequence'].apply(np.array).reset_index(name='stops_sequence')
# 3s - 5s, using np.array. Faster than using list (1m 30s - 4m)
df4_trips = pd.merge(df4_trips1, df4_trips2, on='trip_id')
df4_trips = pd.merge(df4_trips, DF[vid]['4']['trips'][['trip_id', 'direction_id']], on='trip_id', how='left')
df4_trips = pd.merge(df4_trips, DF[vid]['4']['trip_ids'], on='trip_id', how='left')
df4_trips['pattern'] = df4_trips['stop_ids'].apply(lambda x: '-'.join(x))

In [291]:
def merge_stop_ids_sequence(stop_ids_list):
    stop_ids_max : list = max(stop_ids_list, key=lambda x: len(x)).copy()
    for stop_ids in stop_ids_list:
        old_i = 0
        stack = []
        for stop_id in stop_ids:
            cur_i = old_i
            while cur_i < len(stop_ids_max) and stop_id != stop_ids_max[cur_i]:
                cur_i += 1
            if cur_i >= len(stop_ids_max):
                stack.append(stop_id)
            else:
                stack.extend(stop_ids_max[old_i:cur_i+1])
                old_i = cur_i+1
        stack.extend(stop_ids_max[old_i:])
        stop_ids_max = stack
    return stop_ids_max

def get_true_stop_order_sequence(stop_ids_full: list, stop_ids: list, stops_sequence: list[int]):
    # assert len(stop_id) == len(stop_sequence)
    stop_true_sequence = []
    i = 0
    j = 0
    while i < len(stop_ids_full) and j < len(stop_ids):
        if stop_ids_full[i] == stop_ids[j]:
            new_ix = i + 1
            cur_ix = stops_sequence[j]
            if new_ix < cur_ix:
                stop_true_sequence.append(cur_ix)
                i = cur_ix - 1
            else:
                stop_true_sequence.append(new_ix)
            j += 1
        i += 1
    assert j == len(stop_ids)
    return stop_true_sequence

In [325]:
df4_tripstops_full_all_patterns = df4_trips.drop_duplicates(subset=['route_code', 'direction_id', 'pattern'])
df4_tripstops_full = df4_tripstops_full_all_patterns.groupby(['route_code', 'direction_id'])['stop_ids'].apply(np.array)
df4_tripstops_full = df4_tripstops_full.apply(lambda x: merge_stop_ids_sequence(x))
df4_tripstops_full.rename('stop_ids_full', inplace=True)
df4_tripstops_full = df4_tripstops_full.reset_index()
df4_tripstops_full['stop_ids_full_count'] = df4_tripstops_full['stop_ids_full'].apply(len)
df4_trips_rck = pd.merge(df4_trips, df4_tripstops_full, on=['route_code', 'direction_id'], how='left')
df4_tripstops_full_all_patterns = pd.merge(df4_tripstops_full_all_patterns, df4_tripstops_full, on=['route_code', 'direction_id'], how='left')
df4_tripstops_full_all_patterns['stops_sequence_full'] = df4_tripstops_full_all_patterns.apply(lambda x: get_true_stop_order_sequence(x['stop_ids_full'], x['stop_ids'], x['stops_sequence']), axis=1)
df4_tripstops_full_all_patterns = df4_tripstops_full_all_patterns[['route_code', 'direction_id', 'pattern', 'stop_ids_full', 'stop_ids_full_count', 'stops_sequence_full']] 
df4_trips_2 = pd.merge(df4_trips, df4_tripstops_full_all_patterns, on=['route_code', 'direction_id', 'pattern'], how='left')

In [326]:
df4_tripstops_full

,route_code,direction_id,stop_ids_full,stop_ids_full_count
0,150,0,"[46700, 50137, 50249, 50274, 50247, 50245, 443...",22
1,150,1,"[28543, 50205, 50203, 22201, 50201, 50093, 500...",23
2,151,0,"[46700, 50137, 50135, 49061, 50057, 50059, 443...",21
3,151,1,"[28537, 50205, 50203, 21656, 21658, 21661, 216...",24
4,152,0,"[28539, 47733, 50068, 50066, 27987, 27989, 279...",24
...,...,...,...,...
708,979,1,"[19818, 45457, 45458, 4258, 4260, 4261, 4262, ...",46
709,981,0,"[19821, 3657, 3658, 3659, 3660, 3661, 3662, 36...",58
710,981,1,"[18112, 45576, 51217, 27962, 47236, 47237, 512...",59
711,982,0,"[19820, 39613, 1311, 1312, 1313, 1314, 1315, 1...",61


In [306]:
assert df4_trips_2.apply(lambda x: len(x['stops_sequence']) == len(x['stops_sequence_full']), axis=1).all()

In [320]:
df4_trips_3 = df4_trips_2.explode(['stop_ids', 'stops_sequence', 'stops_sequence_full'])[['trip_id', 'route_code', 'direction_id', 'stop_ids', 'stops_sequence', 'stops_sequence_full']]
# 5s - 10s
df4_trips_3.rename(columns={'stop_ids': 'stop_id', 'stops_sequence': 'stop_sequence', 'stops_sequence_full': 'stop_sequence_real'}, inplace=True)
df4_trips_3 = pd.merge(df4_trips_3, DF[vid]['4']['stop_times'], on=['trip_id', 'stop_id', 'stop_sequence'], how='left')
# 10s - 20s

In [321]:
df4_trips_3


,trip_id,route_code,direction_id,stop_id,stop_sequence,stop_sequence_real,arrival_time,departure_time,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,12-831--1-MF1-104,831,0,28341,1,1,06:36:00,06:36:00,NaN,0,0,0.00
1,12-831--1-MF1-104,831,0,22800,2,2,06:40:00,06:40:00,NaN,0,0,1162.50
2,12-831--1-MF1-104,831,0,14828,3,3,06:41:00,06:41:00,NaN,0,0,2036.78
3,12-831--1-MF1-104,831,0,14829,4,4,06:43:00,06:43:00,NaN,0,0,2941.52
4,12-831--1-MF1-104,831,0,14588,5,5,06:43:00,06:43:00,NaN,0,0,3058.57
...,...,...,...,...,...,...,...,...,...,...,...,...
5584903,82-498--1-Sun4-33174,498,1,46065,24,24,21:53:00,21:53:00,NaN,0,0,12981.70
5584904,82-498--1-Sun4-33174,498,1,16080,25,25,21:54:00,21:54:00,NaN,0,0,13377.64
5584905,82-498--1-Sun4-33174,498,1,16081,26,26,21:55:00,21:55:00,NaN,0,0,13910.28
5584906,82-498--1-Sun4-33174,498,1,22606,27,27,21:56:00,21:56:00,NaN,0,0,14245.81


In [319]:
df4_trips_3

,trip_id,route_code,direction_id,stop_id_x,stop_sequence,stop_sequence_real,arrival_time,departure_time,stop_id_y,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,12-831--1-MF1-104,831,0,28341,1,1,06:36:00,06:36:00,28341,NaN,0,0,0.00
1,12-831--1-MF1-104,831,0,22800,2,2,06:40:00,06:40:00,22800,NaN,0,0,1162.50
2,12-831--1-MF1-104,831,0,14828,3,3,06:41:00,06:41:00,14828,NaN,0,0,2036.78
3,12-831--1-MF1-104,831,0,14829,4,4,06:43:00,06:43:00,14829,NaN,0,0,2941.52
4,12-831--1-MF1-104,831,0,14588,5,5,06:43:00,06:43:00,14588,NaN,0,0,3058.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5584903,82-498--1-Sun4-33174,498,1,46065,24,24,21:53:00,21:53:00,46065,NaN,0,0,12981.70
5584904,82-498--1-Sun4-33174,498,1,16080,25,25,21:54:00,21:54:00,16080,NaN,0,0,13377.64
5584905,82-498--1-Sun4-33174,498,1,16081,26,26,21:55:00,21:55:00,16081,NaN,0,0,13910.28
5584906,82-498--1-Sun4-33174,498,1,22606,27,27,21:56:00,21:56:00,22606,NaN,0,0,14245.81
